In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
df=pd.read_csv(r'C:\Users\khale\OneDrive\Desktop\SA_IITG\hackathon\training_set_features.csv')

In [3]:
cat_cols=['race','hhs_geo_region','census_msa','employment_industry','employment_occupation']
dfc=df.copy()

In [4]:
encoder = OneHotEncoder(sparse=False)
one_hot_encoded = encoder.fit_transform(dfc[cat_cols])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(cat_cols))
df_encoded = pd.concat([dfc, one_hot_df], axis=1)
df_encoded = df_encoded.drop(cat_cols, axis=1)

In [5]:
df_encoded['age_group'].replace(['65+ Years','55 - 64 Years','45 - 54 Years','18 - 34 Years','35 - 44 Years'],[4,3,2,0,1],inplace=True)
df_encoded['sex'].replace(['Female','Male'],[0,1],inplace=True)
df_encoded['rent_or_own'].replace(['Own','Rent'],[1,0],inplace=True)
df_encoded['employment_status'].replace(['Employed','Not in Labor Force','Unemployed'],[2,1,0],inplace=True)
df_encoded['education'].replace(['College Graduate','Some College','12 Years','< 12 Years'],[3,2,1,0],inplace=True)
df_encoded['marital_status'].replace(['Not Married','Married'],[0,1],inplace=True)
df_encoded['income_poverty'].replace(['<= $75,000, Above Poverty','> $75,000','Below Poverty'],[1,2,0],inplace=True)

In [6]:
col=df_encoded.columns.to_list()
df_encoded[col] = df_encoded[col].fillna(df_encoded[col].median())
df_encoded=df_encoded.drop(['respondent_id','health_insurance'],axis=1)

In [7]:
X_train=df_encoded
y=pd.read_csv(r'C:\Users\khale\OneDrive\Desktop\SA_IITG\hackathon\training_set_labels.csv')
y=y.drop(['respondent_id'],axis=1)

In [15]:
X_test=pd.read_csv(r'C:\Users\khale\OneDrive\Desktop\SA_IITG\hackathon\test_set_features.csv')

In [16]:
one_hot_encoded = encoder.fit_transform(X_test[cat_cols])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(cat_cols))
df_encoded2 = pd.concat([X_test, one_hot_df], axis=1)
df_encoded2 = df_encoded2.drop(cat_cols, axis=1)

df_encoded2['sex'].replace(['Female','Male'],[0,1],inplace=True)
df_encoded2['rent_or_own'].replace(['Own','Rent'],[1,0],inplace=True)
df_encoded2['employment_status'].replace(['Employed','Not in Labor Force','Unemployed'],[2,1,0],inplace=True)
df_encoded2['education'].replace(['College Graduate','Some College','12 Years','< 12 Years'],[3,2,1,0],inplace=True)
df_encoded2['marital_status'].replace(['Not Married','Married'],[0,1],inplace=True)
df_encoded2['income_poverty'].replace(['<= $75,000, Above Poverty','> $75,000','Below Poverty'],[1,2,0],inplace=True)
df_encoded2['age_group'].replace(['65+ Years','55 - 64 Years','45 - 54 Years','18 - 34 Years','35 - 44 Years'],[4,3,2,0,1],inplace=True)


In [17]:
df_encoded2[col] = df_encoded2[col].fillna(df_encoded2[col].median())

In [18]:
df_encoded2=df_encoded2.drop(['respondent_id','health_insurance'],axis=1)

In [67]:
from sklearn.linear_model import LogisticRegression
# instantiate the model (using the default parameters)

model= LogisticRegression(max_iter=1000,random_state=3)
# fit the model with data
model.fit(X_train, y['seasonal_vaccine'])
y_pred1 = model.predict_proba(df_encoded2)

In [68]:
model.fit(X_train,y['xyz_vaccine'])
y_pred2=model.predict_proba(df_encoded2)

In [69]:
y_pred1=y_pred1[:,0]
y_pred2=y_pred2[:,0]


In [70]:
res_id=X_test.iloc[:,0:1]

In [71]:
y_pred2=pd.DataFrame({ 
    "h1n1_vaccine":y_pred2
}
)
y_pred2

,h1n1_vaccine
0,0.943239
1,0.957745
2,0.611743
3,0.512779
4,0.837295
...,...
26703,0.641173
26704,0.911495
26705,0.872015
26706,0.935901


In [72]:
y_pred1=pd.DataFrame({ 
    "seasonal_vaccine":y_pred1
}
)
y_pred1

,seasonal_vaccine
0,0.742310
1,0.952206
2,0.386545
3,0.142045
4,0.557253
...,...
26703,0.453605
26704,0.686362
26705,0.813889
26706,0.567944


In [78]:
ans=pd.concat([res_id,y_pred2,y_pred1],axis=1)


,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.943239,0.742310
1,26708,0.957745,0.952206
2,26709,0.611743,0.386545
3,26710,0.512779,0.142045
4,26711,0.837295,0.557253
...,...,...,...
26703,53410,0.641173,0.453605
26704,53411,0.911495,0.686362
26705,53412,0.872015,0.813889
26706,53413,0.935901,0.567944


In [79]:
ans.to_csv('ans.csv')